# 03. Mock Photon Comparison

In this Notebook we compare the mock photon propagation with the existing methods like the normal flow. To do that, we generate a line detector and a couple of events that we propagate using each propagator.

In [1]:
import pandas as pd

from ananke.configurations.detector import DetectorConfiguration
from ananke.services.detector import DetectorBuilderService
from olympus.event_generation.generators import GeneratorCollection, GeneratorFactory
from olympus.event_generation.medium import MediumEstimationVariant, Medium
from olympus.event_generation.photon_propagation.mock_photons import MockPhotonPropagator
from olympus.event_generation.photon_propagation.norm_flow_photons import NormalFlowPhotonPropagator

Let's define the detector next:

In [54]:
oms_per_line = 20
dist_z = 50  # m
dark_noise_rate = 16 * 1e-5  # 1/ns
side_len = 100  # m
pmts_per_module = 16
pmt_cath_area_r = 75e-3 / 2  # m
module_radius = 0.21  # m
efficiency = 0.42 # Christian S. Number

detector_configuration = DetectorConfiguration.parse_obj(
    {
        "string": {
            "module_number": oms_per_line,
            "module_distance": dist_z
        },
        "pmt": {
            "efficiency": efficiency,
            "noise_rate": dark_noise_rate,
            "area": pmt_cath_area_r
        },
        "module": {
            "radius": module_radius
        },
        "geometry": {
            "type": "single",
        },
        "seed": 31338
    }
)

detector_service = DetectorBuilderService()
det = detector_service.get(configuration=detector_configuration)

In [33]:
det.df[['module_location_x', 'module_location_y', 'module_location_z']].drop_duplicates()

,module_location_x,module_location_y,module_location_z
0,0.0,0.0,0.0
16,0.0,0.0,50.0
32,0.0,0.0,100.0
48,0.0,0.0,150.0
64,0.0,0.0,200.0
80,0.0,0.0,250.0
96,0.0,0.0,300.0
112,0.0,0.0,350.0
128,0.0,0.0,400.0
144,0.0,0.0,450.0


Next up we generate our events:

In [45]:
medium = Medium(MediumEstimationVariant.PONE_OPTIMISTIC)

generator_factory = GeneratorFactory(det, seed=3118)

cascades_generator = generator_factory.create(
    "cascade", particle_id=11, log_minimal_energy=2, log_maximal_energy=5.5, rate=0.05
)

records = cascades_generator.generate_records(
    number_of_samples=2
)

sources = cascades_generator.propagate(records)

records.df.head()

,location_x,location_y,location_z,orientation_x,orientation_y,orientation_z,record_id,energy,length,time,type,particle_id
0,48.477898,-12.170243,-211.414151,0.906723,-0.004053,-0.421707,8860744248291955181,118401.349187,3000.0,0.0,cascade,11
1,-43.556438,0.797926,-87.918515,0.862624,-0.003856,0.505831,8860745107285414381,446.013358,3000.0,0.0,cascade,11


In [46]:
sources.df.head()

,location_x,location_y,location_z,orientation_x,orientation_y,orientation_z,record_id,time,number_of_photons,type
0,48.526947,-12.026059,-211.549576,0.906723,-0.004053,-0.421707,8860744248291955181,0.336900,1,cherenkov
1,48.748787,-12.056716,-211.546570,0.906723,-0.004053,-0.421707,8860744248291955181,1.004028,907,cherenkov
2,48.953278,-12.305337,-211.578766,0.906723,-0.004053,-0.421707,8860744248291955181,1.671156,28867,cherenkov
3,49.171074,-12.327730,-211.584518,0.906723,-0.004053,-0.421707,8860744248291955181,2.338284,294148,cherenkov
4,49.318699,-11.985676,-211.744644,0.906723,-0.004053,-0.421707,8860744248291955181,3.005413,1602507,cherenkov


Now we need our photon propagators

In [55]:
mock_photon_propagator = MockPhotonPropagator(
    detector=det,
    medium=medium,
    angle_resolution=18000,
)

normal_photon_propagator = NormalFlowPhotonPropagator(
    detector=det,
    shape_model_path="../../hyperion/data/photon_arrival_time_nflow_params.pickle",
    counts_model_path="../../hyperion/data/photon_arrival_time_counts_params.pickle",
    medium=medium
)

Now lets Propagate:

In [56]:
mock_hits = mock_photon_propagator.propagate(records, sources)

mock_hits.df.head()

,time,string_id,module_id,pmt_id,record_id
0,7.446158,0,0,0,8860745107285414381
1,7.625569,0,0,0,8860745107285414381
2,18.417603,0,0,0,8860745107285414381
3,16.245899,0,0,0,8860745107285414381
4,10.487187,0,0,0,8860745107285414381


And doing the same with the normal photons.

In [57]:
normal_hits = normal_photon_propagator.propagate(records, sources)

normal_hits.df.head()

,time,string_id,module_id,pmt_id,record_id
0,70.154221,0,0,1,8860744248291955181
0,154.430084,0,0,2,8860744248291955181
0,25.088919,0,0,3,8860744248291955181
1,27.700706,0,0,3,8860744248291955181
0,477.286163,0,0,5,8860744248291955181


## Comparison of the photon propagators

Now that we have all the hits we want we can compare the following cases:

1. number of hits per module
2. hit arrival times


### Number of hits per module

Let's have a look at the mock propagation:

In [58]:
aggregated_mock_hits = mock_hits.df.set_index(['string_id', 'module_id'])
aggregated_mock_hits.groupby(level=[0,1]).count().head()

time  pmt_id  record_id
string_id module_id                         
0         0            71      71         71
          1            65      65         65

In [59]:
aggregated_normal_hits = normal_hits.df.set_index(['string_id', 'module_id'])
aggregated_normal_hits.groupby(level=[0,1]).count().head()

time  pmt_id  record_id
string_id module_id                         
0         0            35      35         35
          1            15      15         15
          2             3       3          3

In [60]:
len(mock_hits)

136

In [61]:
len(normal_hits)

53